In [ ]:
# Takes in a given comparison file (in the correct format), and calculates the KCRV and DoEs using the largest consistent
# subset. Returns an excel file with the measurements, uncertainties, participants included, DoEs and KCRVs.

# Need to specify the directory where the comparison files are saved. The output files are saved in the same directory that the
# code is being run from.

In [5]:
import xlrd
import math
import xlwt
from scipy import stats
import numpy as np
from scipy.stats import chi2
import operator as op
import itertools

In [28]:
# Run the methods and read data files so we can call those functions
%run "Methods-Using-GLS.ipynb" -G

In [36]:
# Specify the directory in which the comparison data files are saved
# List all files which you want to analyse
files = ["CCL-K1-1.xls"]